In [12]:
import pandas as pd

# Configuration
INPUT_PATH = "/Users/vighneshms/Downloads/Email_classifier/models/combined_emails_with_natural_pii.csv"
OUTPUT_PATH = "/Users/vighneshms/Downloads/Email_classifier/models/emails_with_subjects.csv"
MISSING_SUBJECT_TOKEN = "[NO_SUBJECT]"

def extract_subject(email):
    """Extract subject with placeholder for missing subjects"""
    if not isinstance(email, str):
        return MISSING_SUBJECT_TOKEN
    
    # Case-insensitive subject detection
    email_lower = email.lower()
    if "subject:" in email_lower[:100]:  # Only check first 100 chars
        subject_marker_index = email_lower.find("subject:") + len("subject:")
        subject = email[subject_marker_index:].split('\n', 1)[0].strip()
        return subject if subject else MISSING_SUBJECT_TOKEN
    
    return MISSING_SUBJECT_TOKEN

# Read original data
print("Reading source data...")
df = pd.read_csv(INPUT_PATH)

# Create new dataframe with required columns
print("Processing emails...")
new_df = pd.DataFrame({
    'subject': df['email'].apply(extract_subject),
    'email': df['email'],  # Keep original complete email
    'type': df['type']
})

# Verification
total_emails = len(new_df)
missing_subjects = sum(new_df['subject'] == MISSING_SUBJECT_TOKEN)
print(f"\nProcessing complete:")
print(f"Total emails: {total_emails}")
print(f"Emails with '{MISSING_SUBJECT_TOKEN}': {missing_subjects} ({missing_subjects/total_emails:.1%})")

# Save to new CSV
new_df.to_csv(OUTPUT_PATH, index=False)
print(f"\nSaved to: {OUTPUT_PATH}")
print("Final columns:", new_df.columns.tolist())

# Show sample
print("\nSample data:")
print(new_df.head(3))

Reading source data...
Processing emails...

Processing complete:
Total emails: 24000
Emails with '[NO_SUBJECT]': 1929 (8.0%)

Saved to: /Users/vighneshms/Downloads/Email_classifier/models/emails_with_subjects.csv
Final columns: ['subject', 'email', 'type']

Sample data:
                                             subject  \
0  Unvorhergesehener Absturz der Datenanalyse-Pla...   
1                           Customer Support Inquiry   
2                      Data Analytics for Investment   

                                               email      type  
0  Subject: Unvorhergesehener Absturz der Datenan...  Incident  
1  Subject: Customer Support Inquiry\n\nSeeking i...   Request  
2  Subject: Data Analytics for Investment\n\nI am...   Request  


In [10]:
# Add column to flag emails where subject wasn't extracted
df['subject_extracted'] = df['subject'] != ""

# Save verification report
verification_path = output_path.replace(".csv", "_verification.csv")
df.to_csv(verification_path, index=False)
print(f"\nFull verification data saved to: {verification_path}")


Full verification data saved to: /Users/vighneshms/Downloads/Email_classifier/models/emails_subject_body_type2_verification.csv


In [7]:
pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=81b0ae2790832e08c08cabebb44e8167306fd12eed2d312191215a83dea46a4e
  Stored in directory: /Users/vighneshms/Library/Caches/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langdetect import detect, LangDetectException
import pandas as pd

def detect_language(text):
    try:
        return detect(text) if pd.notna(text) and text.strip() else 'unknown'
    except LangDetectException:
        return 'unknown'

# Detect language for each email
df['language'] = df['body'].apply(detect_language)

# Check distribution
print(df['language'].value_counts())

language
en    15321
de     6897
es      810
fr      484
pt      475
nl       12
it        1
Name: count, dtype: int64


In [13]:
pip install torch transformers pandas tqdm deep-translator langdetect

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
from tqdm import tqdm
import time
from threading import Lock
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Configuration
INPUT_PATH = "/Users/vighneshms/Downloads/Email_classifier/models/emails_with_subjects.csv"
OUTPUT_PATH = "/Users/vighneshms/Downloads/Email_classifier/models/emails_translated.csv"
MISSING_SUBJECT_TOKEN = "[NO_SUBJECT]"
DEVICE = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Initialize counters and lock for thread safety
counters = {
    'total': 0,
    'translated': 0,
    'skipped': 0,
    'failed': 0,
    'english': 0
}
counter_lock = Lock()

# Initialize translation model for MPS
print("Loading translation model...")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-mul-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en").to(DEVICE)

def update_counter(key, value=1):
    with counter_lock:
        counters[key] += value
        # Live display
        print(f"\rTotal: {counters['total']} | "
              f"Translated: {counters['translated']} | "
              f"English: {counters['english']} | "
              f"Skipped: {counters['skipped']} | "
              f"Failed: {counters['failed']}", 
              end="", flush=True)

def mps_translate(text, max_length=512):
    """Translate using MPS-accelerated model"""
    if not isinstance(text, str) or not text.strip() or text == MISSING_SUBJECT_TOKEN:
        return text
    
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length).to(DEVICE)
        outputs = model.generate(**inputs)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    except Exception as e:
        print(f"\nTranslation error: {str(e)}")
        return text

def translate_text(text):
    """Hybrid translation approach with live counting"""
    update_counter('total')
    
    # Skip empty or placeholder texts
    if not isinstance(text, str) or not text.strip() or text == MISSING_SUBJECT_TOKEN:
        update_counter('skipped')
        return text
    
    # Check if already English
    try:
        if detect(text) == 'en':
            update_counter('english')
            return text
    except LangDetectException:
        pass
    
    # Try MPS translation first
    translated = mps_translate(text)
    if translated != text:
        update_counter('translated')
        return translated
    
    # Fallback to Google Translate if MPS fails
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        if translated:
            update_counter('translated')
            return translated
        return text
    except Exception as e:
        update_counter('failed')
        print(f"\nGoogle translate failed: {str(e)}")
        return text

# Read data
print("\nLoading data...")
df = pd.read_csv(INPUT_PATH)

# Process with MPS acceleration
print("\nStarting translation (MPS accelerated)...")
tqdm.pandas(desc="Overall progress")
df['subject_en'] = df['subject'].progress_apply(translate_text)
df['email_en'] = df['email'].progress_apply(translate_text)

# Save results
output_cols = ['subject', 'subject_en', 'email', 'email_en', 'type']
df[output_cols].to_csv(OUTPUT_PATH, index=False)

print(f"\n\nTranslation complete! Saved to: {OUTPUT_PATH}")
print("Final counts:")
print(f"Total processed: {counters['total']}")
print(f"Successfully translated: {counters['translated']}")
print(f"Already English: {counters['english']}")
print(f"Skipped: {counters['skipped']}")
print(f"Failed: {counters['failed']}")

Loading translation model...


/Users/vighneshms/Library/Python/3.9/lib/python/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



Loading data...

Starting translation (MPS accelerated)...


Overall progress:   0%|          | 0/24000 [00:00<?, ?it/s]

Total: 1 | Translated: 1 | English: 0 | Skipped: 0 | Failed: 0

...

Total: 1019 | Translated: 311 | English: 659 | Skipped: 49 | Failed: 0

Overall progress:   4%|▍         | 1020/24000 [03:09<1:36:37,  3.96it/s]

Total: 1020 | Translated: 311 | English: 659 | Skipped: 49 | Failed: 0

Overall progress:   4%|▍         | 1020/24000 [03:09<1:11:20,  5.37it/s]


KeyboardInterrupt: 